In [11]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
from util import management as mana
from util import constants as con
from util import data

def load_summary_dices_data_dict(path):
    summary = data.load_summary_data(path)

    # Load data
    dices = []
    for val in summary:
        dice = val["1"]["Dice"] # pancreas label == 1
        ref = val["reference"].split("/")[-1]
        dices.append((ref, dice))
    return dices

def get_worst(equal, not_equal, task, config, trainer, checkVal=False, amount=10):
    # Load data
    if checkVal:
        path, _ = data.get_results_validation_and_test_paths(con.RESULTS_DIR, task, config, trainer) 
    else:
        _, path = data.get_results_validation_and_test_paths(con.RESULTS_DIR, task, config, trainer) 

    # get summary.json
    summary = data.load_summary_data(path)

    # Load data
    res = []
    for val in summary:
        dice = val["1"]["Dice"] # pancreas label == 1
        ref = val["reference"].split("/")[-1]
        res.append((ref, dice))

    # Print the 10 worst
    res.sort(key=lambda s: s[1])
    worst = [i[0] for i in res][:amount]
    if len(equal) == 0:
        equal.update(worst)
    else:
        not_equal.update(equal.symmetric_difference(set(worst)))
        equal = equal.intersection(worst)
    return equal, not_equal

c:\Users\ikke_\OneDrive\Documenten\Thesis\MasterThesis\Code


Experiment 1

In [4]:
trainer = con.CLASSIC
folds = [0,1,2,3,4]
labels = [1]

for task in [con.TASK_501, con.TASK_510]:
    equal = set()
    not_equal = set()
    for config in [con.CONFIG_LOW, con.CONFIG_CAS_FULL, con.CONFIG_FULL]:
        equal, not_equal = get_worst(equal, not_equal, task, config, trainer)

    # Add cropping
    equal, not_equal = get_worst(equal, not_equal, con.CROP_TASK_MAPPING[task], con.CONFIG_FULL, trainer)

    # Final result
    print(task)
    print(sorted(equal))
    print(sorted(not_equal))
    print()

501
['panc_0001.nii.gz', 'panc_0009.nii.gz', 'panc_0033.nii.gz', 'panc_0041.nii.gz', 'panc_0049.nii.gz', 'panc_0065.nii.gz', 'panc_0073.nii.gz', 'panc_0081.nii.gz', 'panc_0082.nii.gz']
['panc_0017.nii.gz', 'panc_0026.nii.gz']

510
['panc_006.nii.gz', 'panc_012.nii.gz', 'panc_064.nii.gz', 'panc_071.nii.gz', 'panc_099.nii.gz', 'panc_145.nii.gz', 'panc_334.nii.gz', 'panc_346.nii.gz']
['panc_042.nii.gz', 'panc_092.nii.gz', 'panc_213.nii.gz', 'panc_218.nii.gz', 'panc_391.nii.gz']



Experiment 2

In [12]:
config = con.CONFIG_FULL
folds = [0,1,2,3,4]
labels = [1]

for task in [con.TASK_525, con.TASK_526, con.TASK_527]:
    equal = set()
    not_equal = set()
    for trainer in [con.CLASSIC, con.WEIGHT01, con.WEIGHT05, con.WEIGHT09]:
        if task == con.TASK_525 and trainer != con.CLASSIC:
            continue
        equal, not_equal = get_worst(equal, not_equal, task, config, trainer, checkVal=True, amount=4)

    # Final result
    print(task)
    print(sorted(equal))
    print(sorted(not_equal))
    print()

525
['panc_0004.nii.gz', 'panc_0007.nii.gz', 'panc_0023.nii.gz', 'panc_0025.nii.gz']
[]

526
['panc_0001.nii.gz', 'panc_0007.nii.gz', 'panc_0009.nii.gz']
['panc_0004.nii.gz', 'panc_0026.nii.gz']

527
['panc_0009.nii.gz', 'panc_0025.nii.gz']
['panc_0004.nii.gz', 'panc_0007.nii.gz', 'panc_0023.nii.gz', 'panc_0026.nii.gz']

